In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import all the requirements Files
import numpy as np
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
#Testing the Train CSV file 
df_train = pd.read_csv('../input/digit-recognizer/train.csv')
df_train.head(10)

In [ ]:
df_train.info

In [ ]:
df_train.shape

In [ ]:
df_train.isnull().sum()

In [ ]:
X_train_orig = df_train.drop('label', axis=1).to_numpy()
X_train_orig

In [ ]:
y_train_orig = df_train.label.to_numpy()
y_train_orig

In [ ]:
#Testing the Test CSV files 
df_test = pd.read_csv('../input/digit-recognizer/test.csv')
df_test.head(10)

In [ ]:
X_test_orig = df_test.to_numpy()
X_test_orig

In [ ]:
X_train = X_train_orig.astype('float32')/255
y_train = y_train_orig.astype('float32')
X_test = X_test_orig.astype('float32')/255

In [ ]:
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

In [ ]:
y_train = tf.one_hot(y_train, 10)
y_train

In [ ]:
print('X_train shape: ',X_train.shape)
print('X_test shape: ',X_test.shape)
print('y_train shape: ',y_train.shape)

In [ ]:
input_shape = (28, 28, 1)

In [ ]:
CNN = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), strides=1, padding='same',input_shape=input_shape),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), strides=1, padding='same'),
    tf.keras.layers.BatchNormalization(axis=3),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same'),
    tf.keras.layers.BatchNormalization(axis=3),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same'),
    tf.keras.layers.BatchNormalization(axis=3),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='same'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=10, activation='softmax'),    
])

In [ ]:
CNN.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
CNN.summary()

In [ ]:
#checking the GPU
!nvidia-smi

In [ ]:
CNN_history = CNN.fit(X_train, y_train, epochs=25)

In [ ]:
df_loss_acc = pd.DataFrame(CNN_history.history)
df_acc = df_loss_acc[['accuracy']]
df_acc.plot(title='CNN Accuracy', figsize=(12, 8)).set(xlabel='Epoch', ylabel='Accuracy')

In [ ]:
df_loss = df_loss_acc[['loss']]

In [ ]:
df_loss.plot(title='CNN Loss', figsize=(12, 8)).set(xlabel='Epoch', ylabel='Loss')

In [ ]:
predictions = CNN.predict(X_test)

In [ ]:
results = np.argmax(predictions,axis=1)
results = pd.Series(results, name="Label")
list=[]
[list.append(i) for i in range(1,28001)]
ImageID = pd.Series(list, name="ImageID").astype("int32")
submission = pd.concat([ImageID,results],axis = 1)

submission.to_csv("BasedCNN Digit-Recoginer.csv", index=False)